In [1]:
import pandas as pd 
import numpy as np
import os  

In [2]:

# Process all the data to get a general format for the graduates people

# Global Variables
general_path  = os.path.abspath(os.path.join(os.getcwd()))
path_preprocessing_data = os.path.abspath(os.path.join(general_path,"Preprocessing_Data"))

# DataFrames

df_preprocessing = pd.DataFrame()


In [3]:
for file in os.listdir(path_preprocessing_data):
    temp_json_file = pd.read_json(os.path.abspath(os.path.join(path_preprocessing_data,file)))

    print(temp_json_file.columns)

Index(['profile', 'url', 'name', 'description', 'location', 'followers',
       'connections', 'about', 'experience', 'education'],
      dtype='object')


In [42]:
temp_json_file['experience'][0]

[{'organisation_profile': 'https://www.linkedin.com/company/turingbox',
  'location': '',
  'description': '',
  'start_time': 'Aug 2022',
  'end_time': 'Apr 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/universidadnacionaldecolombia',
  'location': 'Bogotá, Distrito Capital, Colombia',
  'description': 'Se desarrolló un análisis de datos de las tesis de posgrado de la Facultad de ingeniería.',
  'start_time': 'May 2022',
  'end_time': 'Sep 2022',
  'duration': None},
 {'organisation_profile': 'https://www.linkedin.com/company/out-xvii',
  'location': '',
  'description': ''}]

In [6]:
temp_json_file['about'][0]#[0]#['about']#.keys()#.unique()#.info()

In [ ]:
# Profile Name -> name (string)
# Profile URL -> url (string)
# Description -> description (string)
# Location (house) -> location (string)
# Education -> (dict) -> organisation_name, organisation_profile
    

In [41]:
import json

def extract_keys(json_object, parent_key='', keys_list=[]):
    if isinstance(json_object, dict):
        for key in json_object.keys():
            new_key = f"{parent_key}.{key}" if parent_key else key
            extract_keys(json_object[key], new_key, keys_list)
    elif isinstance(json_object, list):
        for index, item in enumerate(json_object):
            new_key = f"{parent_key}[{index}]"
            extract_keys(item, new_key, keys_list)
    else:
        keys_list.append(parent_key)
    return keys_list

# Sample JSON data (replace this with your actual JSON data)
json_data = {"first_name":"Juan Sebastian","last_name":"Martinez Beltr\u00e1n","full_name":"Juan Sebastian Martinez Beltr\u00e1n","gender":"male",
             "job_title":"Student in Universidad Nacional de Colombia",
             "linkedin":"https://www.linkedin.com/in/juan-sebasti\u00e1n-mart\u00ednez-beltr\u00e1n-5a1123150",
             "summary":"In my academic history i have been class president in the last grade in High School. To fulfill my high school education i made social service in Aspacafam, working 80 hours doing office duties and taking care of children in a country club.\n\nFourth place out of a thousand in the respective group of the ICFES results in September 2nd 2012. Fifth Place in the admission test for the Colombian National University for the semester of 2013-2. And my academic grade point average for the whole career is 4,3 out of 5,0\n\nFourth place along with my work group in the 12th version of the project exhibition event event \"EXPOIDEAS\" from the Universidad Nacional de Colombia, made in May 18th, 2016.\n\nSelected as scholar/prefect in the first semester of 2018 in the Universidad Nacional de Colombia, where i served as prefect in the subject of \"Operating Systems\" and assistant in the development of web pages for the University\u2019s Faculty of Engineering.\n\nCurrently working at Geekcore S.A.S. as a web developer, focused in front-end development.\n\nDuring 7 months i was working in Geekcore S.A.S. as a web developer focused in Front End development, not only doing programming tasks, but also documentation making, like user manuals, architecture description documents and IT management.\n\nDuring 6 months I was an Intern in Huawei Technologies CO. LTD. where I was in charge of the management of projects, contracts and quotation related to technology projects. At the end of my period, I won an \"Excelence Certificate\" for Best Performance as an intern in a group of 12.\n\nCurrently I'm working as a FrontEnd developer for S4N S.A.S. For six months I worked for Lever in a FullStack envoriment based on NodeJS, CoffeeScript and TypeScript. I'm currently focusing in the use of ReactJS and Hooks, working for Copa Airlines.\n\n\nMy hobbies mostly consist in the production of multimedia, like videos and 3D modeling. Writting is also one of my greatest hobbies, and where i feel my biggest achievement is, during 5 years i wrote a novel of over 850 pages, that is currently in home made editing.","premium":False,"skills":"Software, Programaci\u00f3n, Ingenier\u00eda inform\u00e1tica, Desarrollo del front-end, Front-end Development, Programming, Software Development, Contract Recruitment, Sales, User Interface Design, Ruby on Rails, Java, Python, SQL, Linux, React.js, HTML, HTML5, CSS, JavaScript","work_experience":[{"company":{"id":4972,"name":"EPAM Systems","logo":"https://media.licdn.com/dms/image/C4D0BAQEE73wQBVodMQ/company-logo_400_400/0/1677598469595?e=1705536000&v=beta&t=yFCT-lCudcvDJ7hbxR-bcFI-VomR_qa9RXhDlu9qBgs","url":"https://www.linkedin.com/company/epam-systems/","employees":{"start":10001,"end":None}},
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               "date":{"start":{"month":9,"day":None,"year":2021},"end":{"month":None,"day":None,"year":None}},"profile_positions":[{"location":"Colombia","date":{"start":{"month":9,"day":None,"year":2021},"end":{"month":None,"day":None,"year":None}},
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     "company":"EPAM Systems","description":None,"title":"Frontend Developer","employment_type":"Full-time"}]},{"company":{"id":390001,"name":"S4N","logo":"https://media.licdn.com/dms/image/C560BAQH0Q0HdGBnEvw/company-logo_400_400/0/1654641889201?e=1705536000&v=beta&t=3pXj3uuYkiyxzcX8qmI5uQqxSZeVtd0RUbpgoiitZCM",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 "url":"https://www.linkedin.com/company/s4n/","employees":{"start":10001,"end":None}},"date":{"start":{"month":10,"day":None,"year":2019},"end":{"month":None,"day":None,"year":None}},"profile_positions":[{"location":"Bogot\u00e1 D.C. Area, Colombia","date":{"start":{"month":10,"day":None,"year":2019},"end":{"month":None,"day":None,"year":None}},"company":"S4N, an EPAM Company","description":None,"title":"Frontend Web Developer","employment_type":"Full-time"}]},{"company":{"id":3014,"name":"Huawei Technologies","logo":"https://media.licdn.com/dms/image/C510BAQEtZ01_ey5nIA/company-logo_400_400/0/1586016478298?e=1705536000&v=beta&t=rY77DrYbRohBByIfItwzl02LOnwsvASqckCRsU_e8iw","url":"https://www.linkedin.com/company/huawei/","employees":{"start":10001,"end":None}},"date":{"start":{"month":4,"day":None,"year":2019},"end":{"month":9,"day":None,"year":2019}},"profile_positions":[{"location":"Bogot\u00e1 D.C. Area, Colombia","date":{"start":{"month":4,"day":None,"year":2019},"end":{"month":9,"day":None,"year":2019}},"company":"Huawei","description":None,"title":"Solution Service Engineer Internship","employment_type":"Internship"}]},{"company":{"id":11506643,"name":"Geekcore","logo":"https://media.licdn.com/dms/image/C4E0BAQFMTLAm-v0EwA/company-logo_400_400/0/1587046169854?e=1705536000&v=beta&t=BtgYa8LEG_K-unhWR_9oi8DhqMmABhqkqkd-aFRm1Qs","url":"https://www.linkedin.com/company/geekcore/","employees":{"start":11,"end":50}},"date":{"start":{"month":8,"day":None,"year":2018},"end":{"month":3,"day":None,"year":2019}},"profile_positions":[{"location":"Bogot\u00e1 D.C. Area, Colombia","date":{"start":{"month":8,"day":None,"year":2018},"end":{"month":3,"day":None,"year":2019}},"company":"Geekcore","description":None,"title":"Junior Developer","employment_type":None}]},{"company":{"id":464241,"name":"Universidad Nacional de Colombia","logo":"https://media.licdn.com/dms/image/D4E0BAQH2fxFDcpvbGw/company-logo_400_400/0/1689003876228?e=1705536000&v=beta&t=oVQVqLNBVhz0_4o5t3feqTSjZMfyfQMMF5QywUGGj8I","url":"https://www.linkedin.com/school/universidadnacionaldecolombia/","employees":{"start":5001,"end":10000}},"date":{"start":{"month":2,"day":None,"year":2018},"end":{"month":6,"day":None,"year":2018}},"profile_positions":[{"location":"Bogot\u00e1 D.C. Area, Colombia","date":{"start":{"month":2,"day":None,"year":2018},"end":{"month":6,"day":None,"year":2018}},"company":"Universidad Nacional de Colombia","description":"Prefect in the Subject of Operating System. Assistant for the develop of Web Pages for the Faculty of Engineering in the \"Universidad Nacional De Colombia\" website","title":"Prefect/Scholar","employment_type":None}]}],"education":[{"date":{"start":{"month":None,"day":None,"year":2013},"end":{"month":None,"day":None,"year":2019}},"school":{"name":"Universidad Nacional de Colombia","logo":"https://media.licdn.com/dms/image/D4E0BAQH2fxFDcpvbGw/company-logo_400_400/0/1689003876228?e=1705536000&v=beta&t=oVQVqLNBVhz0_4o5t3feqTSjZMfyfQMMF5QywUGGj8I","url":"https://www.linkedin.com/school/universidadnacionaldecolombia/"},"degree_name":"Bachelor's degree","field_of_study":"Systems Engineering","grade":None}],"languages":{"primary_locale":{"country":"ES","language":"es"},"supported_locales":[{"country":"ES","language":"es"},{"country":"US","language":"en"}],"profile_languages":[]},"current_job_year":2021,"current_job_month":9,"picture":None,"email":{"email":"juan@epam.com","email_anon_id":"A7alTCryhpYf","email_status":"CATCH_ALL","email_type":"professional"},"company":{"name":"EPAM Systems","is_catch_all":True,"size":"1001-5000","logo":"https://assets-prospeo.s3.us-east-2.amazonaws.com/company_KGMJAKXKFD8L71DCEI1T.jpg","linkedin":"https://www.linkedin.com/company/epam-systems","website":"http://www.epam.com","domain":"epam.com","common_email_pattern":None,"industry":"IT Services and IT Consulting","founded_in":1993,"description":"Since 1993,\u202fEPAM Systems, Inc.\u202f(NYSE: EPAM) has leveraged its advanced software engineering heritage to become the foremost global digital transformation services provider \u2013 leading the industry in digital and physical product development and digital platform engineering services. Through its innovative strategy; integrated advisory, consulting, and design capabilities; and unique 'Engineering DNA,' EPAM's globally deployed hybrid teams help make the future real for clients and communities around the world by powering better enterprise, education and health platforms that connect people, optimize experiences, and improve people's lives. In 2021, EPAM was added to the S&P 500 and included among the list of Forbes Global 2000 companies.\u202f\u202f\n\nSelected by Newsweek as a 2021 and 2022 Most Loved Workplace, EPAM's global multi-disciplinary teams serve customers in more than 50 countries across six continents. As a recognized leader, EPAM is listed among the top 15 companies in Information Technology Services on the Fortune 1000 and ranked four times as the top IT services company on Fortune's 100 Fastest Growing Companies list. EPAM is also listed among Ad Age's top 25 World's Largest Agency Companies for three consecutive years, and Consulting Magazine named EPAM Continuum a top 20 Fastest Growing Firm.\n\nLearn more at\u202f www.epam.com.","total_emails":1000,"location":{"country":"United States","country_code":"US","state":None,"city":"Newtown","timezone":"America/New_York","timezone_offset":"-4.0","postal_code":None,"address":"41 University Drive Suite 202"}},"location":{"country":"Colombia","country_code":"CO","state":None,"city":"Bogota","timezone":"America/Bogota","timezone_offset":"-5.0","postal_code":None,"raw":"Bogota, D.C., Capital District, Colombia"}}

# Convert the JSON object to a string (if it's not already a string)
json_str = json.dumps(json_data)

# Parse the JSON string to a Python object
parsed_json = json.loads(json_str)

# Extract keys
keys = extract_keys(parsed_json)
print("Extracted Keys:")
for key in keys:
    print(key)

Extracted Keys:
first_name
last_name
full_name
gender
job_title
linkedin
summary
premium
skills
work_experience[0].company.id
work_experience[0].company.name
work_experience[0].company.logo
work_experience[0].company.url
work_experience[0].company.employees.start
work_experience[0].company.employees.end
work_experience[0].date.start.month
work_experience[0].date.start.day
work_experience[0].date.start.year
work_experience[0].date.end.month
work_experience[0].date.end.day
work_experience[0].date.end.year
work_experience[0].profile_positions[0].location
work_experience[0].profile_positions[0].date.start.month
work_experience[0].profile_positions[0].date.start.day
work_experience[0].profile_positions[0].date.start.year
work_experience[0].profile_positions[0].date.end.month
work_experience[0].profile_positions[0].date.end.day
work_experience[0].profile_positions[0].date.end.year
work_experience[0].profile_positions[0].company
work_experience[0].profile_positions[0].description
work_experie